In [36]:
import cv2
import numpy as np
from collections import Counter
import os

print("Imports complete.")

Imports complete.


In [ ]:
# ---------------------------
# Fibonacci Code for a Rank
# ---------------------------
def fib_code_for_rank(r):
    """
    Compute the Fibonacci code for an integer 'r' (r >= 1) using a Fibonacci numeral system.
    Expected examples:
      Rank 1 -> "11"
      Rank 2 -> "011"
      Rank 3 -> "0011"
      Rank 4 -> "1011"
      Rank 5 -> "00011"
      Rank 6 -> "10011"
      etc.
    This implementation builds a Fibonacci sequence and creates a code based on a greedy (Zeckendorf) representation.
    """
    if r < 1:
        raise ValueError("Rank must be >= 1")
    
    # Generate Fibonacci numbers (starting with 1, 2, 3, 5, ...)
    fibs = [1, 2]
    while fibs[-1] < r:
        fibs.append(fibs[-1] + fibs[-2])
    
    # Build the Fibonacci representation (greedy algorithm)
    code_bits = []
    remaining = r
    for f in reversed(fibs):
        if remaining >= f:
            code_bits.append('1')
            remaining -= f
        else:
            if code_bits:  # Only add zeros after the first '1'
                code_bits.append('0')
    
    # Reverse the bits so that the code is read from smallest Fibonacci number upward
    code_str = ''.join(code_bits)[::-1]
    
    # Pad with zeros so the length equals the number of Fibonacci numbers used
    code_str = code_str.zfill(len(fibs))
    
    # Append the termination marker "1"
    code_str += "1"
    return code_str

print("Fibonacci coding function defined.")


In [37]:
# ---------------------------
# Frequency and Code Mapping Functions
# ---------------------------
def get_frequency_table(image):
    """
    Given a single-channel image (numpy array), compute the frequency distribution
    of pixel values. Returns a dictionary {pixel_value: frequency}.
    """
    pixels = image.flatten()
    return dict(Counter(pixels))

def get_code_mapping(freq_table):
    """
    Given a frequency table (dictionary), sort the pixel values by frequency (descending)
    and assign a Fibonacci code based on ranking (starting with rank 1).
    Returns a tuple (code_mapping, sorted_items) where:
      - code_mapping is a dictionary {pixel_value: fibonacci_code}
      - sorted_items is a list of (pixel, frequency) tuples sorted in descending order.
    """
    # Sort by frequency descending; if equal, sort by pixel value
    sorted_items = sorted(freq_table.items(), key=lambda x: (-x[1], x[0]))
    code_mapping = {}
    for rank, (pixel, _) in enumerate(sorted_items, start=1):
        code_mapping[pixel] = fib_code_for_rank(rank)
    return code_mapping, sorted_items

print("Frequency and code mapping functions defined.")


Frequency and code mapping functions defined.


In [38]:
# ---------------------------
# Compression and Decompression Functions
# ---------------------------
def compress_image(image, code_mapping):
    """
    Compress a 2D image (single-channel) by replacing each pixel with its corresponding Fibonacci code.
    Returns the compressed bitstream (a string).
    """
    pixels = image.flatten()
    bitstream = "".join(code_mapping[p] for p in pixels)
    return bitstream

def decompress_image(bitstream, code_mapping, image_shape):
    """
    Decompress a bitstream using the provided code mapping.
    The Fibonacci codes are prefix-free so we iterate bit-by-bit and decode when a valid code is found.
    Returns the decompressed image (numpy array) with shape image_shape.
    """
    # Create reverse mapping: code -> pixel value
    reverse_mapping = {v: k for k, v in code_mapping.items()}
    decoded_pixels = []
    buffer = ""
    for bit in bitstream:
        buffer += bit
        if buffer in reverse_mapping:
            decoded_pixels.append(reverse_mapping[buffer])
            buffer = ""
    return np.array(decoded_pixels, dtype=np.uint8).reshape(image_shape)

print("Compression and decompression functions defined.")


Compression and decompression functions defined.


In [39]:
# ---------------------------
# Processing Functions for Single Channel and Color Images
# ---------------------------
def process_single_channel_image(image_path, mode_label):
    """
    Process a single-channel image (for BNW or Grayscale).
    - Loads the image.
    - Computes frequency table.
    - Creates code mapping (Fibonacci codes) based on ranking.
    - Compresses the image into a bitstream.
    - Prints frequency table and compression statistics.
    - Decompresses the image and saves the result.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Could not load {image_path}")
        return
    
    freq_table = get_frequency_table(image)
    code_mapping, sorted_items = get_code_mapping(freq_table)
    bitstream = compress_image(image, code_mapping)
    
    original_bits = image.size * 8  # 8 bits per pixel
    compressed_bits = len(bitstream)
    compression_ratio = compressed_bits / original_bits
    
    print(f"--- {mode_label} Image: {os.path.basename(image_path)} ---")
    print("Pixel Frequency Table (sorted):")
    print("Value | Frequency | Fibonacci Code")
    for pixel, freq in sorted_items:
        print(f"{pixel:5} | {freq:9} | {code_mapping[pixel]}")
    print(f"\nOriginal Size: {original_bits} bits")
    print(f"Compressed Size: {compressed_bits} bits")
    print(f"Compression Ratio: {compression_ratio:.2f} (compressed/original)")
    print(f"Space Savings: {100*(1-compression_ratio):.1f}%\n")
    
    # Decompress the image and save
    decompressed_image = decompress_image(bitstream, code_mapping, image.shape)
    out_path = f"decompressed_{mode_label}_{os.path.basename(image_path)}"
    cv2.imwrite(out_path, decompressed_image)
    print(f"Decompressed image saved as: {out_path}\n")
    
def process_color_image(image_path):
    """
    Process a color image (BGR) by splitting into channels, compressing each channel separately,
    and then merging the decompressed channels back together.
    Prints frequency tables and compression statistics for each channel.
    """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        print(f"Error: Could not load {image_path}")
        return
    channels = cv2.split(image)
    channel_labels = ['B', 'G', 'R']
    overall_original = image.size * 8  # total bits for all channels
    overall_compressed = 0
    decompressed_channels = []
    
    print(f"--- Color Image: {os.path.basename(image_path)} ---")
    for idx, channel in enumerate(channels):
        freq_table = get_frequency_table(channel)
        code_mapping, sorted_items = get_code_mapping(freq_table)
        bitstream = compress_image(channel, code_mapping)
        overall_compressed += len(bitstream)
        original_bits = channel.size * 8
        
        print(f"\nChannel {channel_labels[idx]} Frequency Table (sorted):")
        print("Value | Frequency | Fibonacci Code")
        for pixel, freq in sorted_items:
            print(f"{pixel:5} | {freq:9} | {code_mapping[pixel]}")
        print(f"\nChannel {channel_labels[idx]} - Original Size: {original_bits} bits")
        print(f"Channel {channel_labels[idx]} - Compressed Size: {len(bitstream)} bits")
        print(f"Channel {channel_labels[idx]} - Compression Ratio: {len(bitstream)/original_bits:.2f}")
        
        # Decompress channel
        decompressed_channel = decompress_image(bitstream, code_mapping, channel.shape)
        decompressed_channels.append(decompressed_channel)
    
    overall_ratio = overall_compressed / overall_original
    print(f"\nOverall Color Image - Original Size: {overall_original} bits")
    print(f"Overall Color Image - Compressed Size: {overall_compressed} bits")
    print(f"Overall Compression Ratio: {overall_ratio:.2f} (compressed/original)")
    print(f"Overall Space Savings: {100*(1-overall_ratio):.1f}%\n")
    
    # Merge decompressed channels and save the image
    decompressed_color = cv2.merge(decompressed_channels)
    out_path = f"decompressed_Color_{os.path.basename(image_path)}"
    cv2.imwrite(out_path, decompressed_color)
    print(f"Decompressed color image saved as: {out_path}\n")
    
print("Processing functions defined.")


Processing functions defined.


In [40]:
# ---------------------------
# Main Processing: Define image paths and run compression/decompression
# ---------------------------
bnw_images = [
    r"C:\Users\user\Desktop\Image Compression\media\bnw200.jpg",
    r"C:\Users\user\Desktop\Image Compression\media\bnw300.jpg"
]

grayscale_images = [
    r"C:\Users\user\Desktop\Image Compression\media\grayscale1-50.jpeg",
    r"C:\Users\user\Desktop\Image Compression\media\grayscale2-100.jpeg"
]

color_images = [
    r"C:\Users\user\Desktop\Image Compression\media\color-400.jpeg",
    r"C:\Users\user\Desktop\Image Compression\media\color500.jpg"
]

print("=== Compression Process ===\n")

# Process Black & White images (loaded as grayscale)
for path in bnw_images:
    process_single_channel_image(path, "BNW")

# Process Grayscale images
for path in grayscale_images:
    process_single_channel_image(path, "Grayscale")

# Process Color images
for path in color_images:
    process_color_image(path)

print("=== Processing Complete ===")


=== Compression Process ===

--- BNW Image: bnw200.jpg ---
Pixel Frequency Table (sorted):
Value | Frequency | Fibonacci Code
    8 |    214146 | 11
    6 |    126418 | 011
    5 |     81803 | 0011
    4 |     58910 | 1011
    3 |     56932 | 00011
    2 |     47841 | 10011
  250 |     34239 | 01011
  249 |     33563 | 000011
  251 |     28716 | 100011
    1 |     26304 | 010011
  252 |     23610 | 001011
  248 |     23102 | 101011
    9 |     20127 | 0000011
    7 |     19463 | 1000011
    0 |     19262 | 0100011
  253 |     17142 | 0010011
  254 |     12601 | 1010011
  247 |      9988 | 0001011
  246 |      5868 | 1001011
  255 |      4729 | 0101011
   10 |      3703 | 00000011
  245 |      3205 | 10000011
  244 |      2883 | 01000011
   11 |      2865 | 00100011
   12 |      2619 | 10100011
  243 |      2443 | 00010011
   17 |      2434 | 10010011
   15 |      2327 | 01010011
   18 |      2118 | 00001011
   16 |      2056 | 10001011
  242 |      1889 | 01001011
   14 |      1867 | 0